*The goal of this project is to predict the violent crime rate based upon features of a community.  The Communities and Crime data set is available at  https://archive.ics.uci.edu/dataset/183/communities+and+crime .*

#Step 0: Import the data.

In [1]:
pip install ucimlrepo --quiet

In [2]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from ucimlrepo import fetch_ucirepo

#fetch data
communities_and_crime = fetch_ucirepo(id=183)

# data (as pandas dataframes)
X = communities_and_crime.data.features
y = communities_and_crime.data.targets

info = [communities_and_crime.metadata, communities_and_crime.variables]

# Step 1: Basic Information.

Now check the type of all the data from the documentation of the dataset.

In [53]:
info[1]['type'].unique()

array(['Integer', 'Categorical', 'Continuous'], dtype=object)

In [68]:
categorical_vars = info[1]['name'].loc[info[1]['type'] == 'Categorical']
# print(info[1].loc[info[1]['type'] == 'Categorical'])
# print('\n')
integer_vars = info[1]['name'].loc[info[1]['type'] == 'Integer']
# print(info[1].loc[info[1]['type'] == 'Integer'])
# print('\n')
continuous_vars = info[1]['name'].loc[info[1]['type'] == 'Continuous']
# print(info[1].loc[info[1]['type'] == 'Continuous'])
# print('\n')

In [74]:
print(f'There are {X.shape[1]} explanatory variables.\n- Of these variables there exists {len(continuous_vars)} continuous variables, {len(integer_vars)} integer variables, and {len(categorical_vars)} categorical variables.')

There are 127 explanatory variables.
- Of these variables there exists 110 continuous variables, 17 integer variables, and 1 categorical variables.


#Step 2: Data Processing.

In [78]:
#Check the response variable - Violent Crimes per Population for missing or non float values.
check = lambda x: True if type(x) == float else False
assert y['ViolentCrimesPerPop'].apply(check).sum() == y.shape[0], 'Error y contains non-float entries'
#okay, y is clean.

In [83]:
df = X.replace('?',np.nan) #replacing the "?" entries with np.nan values
print(f'There are {df.shape[1]} explanatory variables each containting {df.shape[0]} entries\n')
df

There are 127 explanatory variables each containting 1994 entries



,state,county,community,communityname,fold,population,householdsize,racepctblack,racePctWhite,racePctAsian,...,PolicAveOTWorked,LandArea,PopDens,PctUsePubTrans,PolicCars,PolicOperBudg,LemasPctPolicOnPatr,LemasGangUnitDeploy,LemasPctOfficDrugUn,PolicBudgPerPop
0,8,NaN,NaN,Lakewoodcity,1,0.19,0.33,0.02,0.90,0.12,...,0.29,0.12,0.26,0.20,0.06,0.04,0.9,0.5,0.32,0.14
1,53,NaN,NaN,Tukwilacity,1,0.00,0.16,0.12,0.74,0.45,...,NaN,0.02,0.12,0.45,NaN,NaN,NaN,NaN,0.00,NaN
2,24,NaN,NaN,Aberdeentown,1,0.00,0.42,0.49,0.56,0.17,...,NaN,0.01,0.21,0.02,NaN,NaN,NaN,NaN,0.00,NaN
3,34,5,81440,Willingborotownship,1,0.04,0.77,1.00,0.08,0.12,...,NaN,0.02,0.39,0.28,NaN,NaN,NaN,NaN,0.00,NaN
4,42,95,6096,Bethlehemtownship,1,0.01,0.55,0.02,0.95,0.09,...,NaN,0.04,0.09,0.02,NaN,NaN,NaN,NaN,0.00,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1989,12,NaN,NaN,TempleTerracecity,10,0.01,0.40,0.10,0.87,0.12,...,NaN,0.01,0.28,0.05,NaN,NaN,NaN,NaN,0.00,NaN
1990,6,NaN,NaN,Seasidecity,10,0.05,0.96,0.46,0.28,0.83,...,NaN,0.02,0.37,0.20,NaN,NaN,NaN,NaN,0.00,NaN
1991,9,9,80070,Waterburytown,10,0.16,0.37,0.25,0.69,0.04,...,0.25,0.08,0.32,0.18,0.08,0.06,0.78,0,0.91,0.28
1992,25,17,72600,Walthamcity,10,0.08,0.51,0.06,0.87,0.22,...,0.19,0.03,0.38,0.33,0.02,0.02,0.79,0,0.22,0.18


Now let us extract the explanatory variables that contain no missing values.

In [84]:
bool_musk = (df.isna().sum() == 0).values
clean_col = list(df.columns[bool_musk])

#Check that they are indeed clean.
for col in clean_col:
  assert df[col].isna().sum() == 0, 'ERROR'

print(f'Out of the {df.shape[1]} there are {len(clean_col)} explanatory variables with no missing values.\nTherefore, {(len(clean_col)/df.shape[1])*100:.2f}% of the explanatory variables are clean; these will be included in our analysis.')

Out of the 127 there are 102 explanatory variables with no missing values.
Therefore, 80.31% of the explanatory variables are clean; these will be included in our analysis.


Our filter of explanatory variables was very naive (if there were missing values we dropped that variable).  Lets go back and check the percentage of data missing in each of the unclean variables.  Furthermore, let us check if any of these variables are such that we would want to include them, e.g. a variable such as *proportion of the community that have a criminal record* should not be easily discarded as this could very well be incredibly significant to the analysis.

Additionally, just because a variable has no missing values does not mean it is ready for analysis.  For example, categorical variables need further processing.  But for now let us filter through the variables that have too many missing values and drop these.

In [63]:
unclean_col = list(df.columns[~bool_musk])

freq_na = df.isna().sum()[unclean_col]

dict_na = {var:[] for var in unclean_col}

for key, val in dict(freq_na).items():
  dict_na[key].append(freq_na[key])
  dict_na[key].append(float(f'{(val / df.shape[0])*100:.3f}'))

df_na = pd.DataFrame(dict_na).T.rename(columns = {0:'NA Count',1:'NA (%)'}).sort_values(by = 'NA (%)')
df_na

,NA Count,NA (%)
OtherPerCap,1.0,0.050
county,1174.0,58.877
community,1177.0,59.027
LemasPctPolicOnPatr,1675.0,84.002
PolicOperBudg,1675.0,84.002
PolicCars,1675.0,84.002
PolicAveOTWorked,1675.0,84.002
NumKindsDrugsSeiz,1675.0,84.002
OfficAssgnDrugUnits,1675.0,84.002
PctPolicMinor,1675.0,84.002


Clearly we should not have dropped *OtherPerCap* as it only has 1 missing entry out of 1994 entries = 5 bps.  This is negligible.  Furthermore, the other variables do not seem to offer enough relevance to include in the model as they have far too high missing data (>50%) and ultimately should be dropped.


Now to replace the single missing value in OtherPerCap.  Since the missing value represents 0.05% of the variables data it is not truly relevant and can be approximated simply by taking the mean as follows.  However, if the missing rate was higher such as 20% and including this variable was of priority another methods such as KNN imputation would be more precise.  This method fills in missing data by estimating them based on the values of the most similar (or nearest) data points. Instead of simply filling in missing values with the mean, median, or mode, KNN uses the patterns in the data to make more informed imputations.

In [85]:
filtered_vars = clean_col + ['OtherPerCap']
print(f'We have {len(filtered_vars)} explanatory variables to use.\n')
#Note 'OtherPerCap has 1 missing value.
#I will take the mean of OtherPerCap and replace the missing value with this.

bool_musk2 = df['OtherPerCap'].isna()
na_index = df['OtherPerCap'].loc[bool_musk2].index[0] #is 130

mean_OPC = ((df['OtherPerCap'].loc[df['OtherPerCap'] != np.nan]).apply(lambda x: float(x))).mean()
median_OPC = ((df['OtherPerCap'].loc[df['OtherPerCap'] != np.nan]).apply(lambda x: float(x))).median()

print(f'The mean of OtherPerCap: {mean_OPC}\nThe median of OtherPerCap: {median_OPC}\n\nUsing the mean to approximate the missing value is viable.')

df.loc[na_index, 'OtherPerCap'] = round(mean_OPC, 4)
df['OtherPerCap'] = pd.to_numeric(df['OtherPerCap'], errors='coerce')

We have 103 explanatory variables to use.

The mean of OtherPerCap: 0.2847415955845459
The median of OtherPerCap: 0.25

Using the mean to approximate the missing value is viable.


Now we drop our variables with high percentages of missing data and further refine the variables that we want to use from the filtered 103 explanatory variables.


In [86]:
X_clean = df[filtered_vars]
X_clean.head()

,state,communityname,fold,population,householdsize,racepctblack,racePctWhite,racePctAsian,racePctHisp,agePct12t21,...,PctForeignBorn,PctBornSameState,PctSameHouse85,PctSameCity85,PctSameState85,LandArea,PopDens,PctUsePubTrans,LemasPctOfficDrugUn,OtherPerCap
0,8,Lakewoodcity,1,0.19,0.33,0.02,0.90,0.12,0.17,0.34,...,0.12,0.42,0.50,0.51,0.64,0.12,0.26,0.20,0.32,0.36
1,53,Tukwilacity,1,0.00,0.16,0.12,0.74,0.45,0.07,0.26,...,0.21,0.50,0.34,0.60,0.52,0.02,0.12,0.45,0.00,0.22
2,24,Aberdeentown,1,0.00,0.42,0.49,0.56,0.17,0.04,0.39,...,0.14,0.49,0.54,0.67,0.56,0.01,0.21,0.02,0.00,0.28
3,34,Willingborotownship,1,0.04,0.77,1.00,0.08,0.12,0.10,0.51,...,0.19,0.30,0.73,0.64,0.65,0.02,0.39,0.28,0.00,0.36
4,42,Bethlehemtownship,1,0.01,0.55,0.02,0.95,0.09,0.05,0.38,...,0.11,0.72,0.64,0.61,0.53,0.04,0.09,0.02,0.00,0.51


Using the information from step 1 we know that there exist three types of data: continuous, categorical, integer types.  The categorical type and integer type contain variables that are not necessarily ready to be used for analysis.  For example, within integer type there exists the variable state.  State has an integer value associated with a specific state but this does not have a numerical relevance.  Therefore, we could apply methods such as one-hot encoding to deal with this issue.  Likewise for the categorical variable communityname.  However, when dealing with a sparse categorical variable like communityname, one-hot encoding can cause serious overfitting. Here's why:

*   High Cardinality Problem: If communityname has many unique values, one-hot encoding will create too many features, making the model complex and harder to generalize.
*   Overfitting on Rare Categories: If a community appears only once, the model may assign a coefficient that fits that single instance perfectly but fails on new data.
*   Increased Computational Cost: More features mean longer training times and more memory usage, which isn't ideal for large datasets.
*   Minimal Predictive Value: The added complexity may not provide useful insights, as each unique community gets its own feature with little generalization power.

Therefore, we should inspect all variables of type integer and categorical and determine if they are relevant and should be included in the final dataset.

In [91]:
print(f'categorical: {categorical_vars.values}')
print(f'integer: {integer_vars.values}')

categorical: ['communityname']
integer: ['state' 'county' 'community' 'fold' 'numbUrban' 'pctUrban' 'NumIlleg'
 'MedOwnCostPctIncNoMtg' 'NumInShelters' 'NumStreet' 'LemasSwFTPerPop'
 'PolicPerPop' 'PctPolicBlack' 'PctPolicHisp' 'LemasGangUnitDeploy'
 'LemasPctOfficDrugUn' 'PolicBudgPerPop']


The variables that are difficult to interpret/implement are communityname, fold, and state.  The others are valid.

In [102]:
X_clean['communityname'].nunique()

1828

To include communityname in the regression we would have to apply one-hot encoding but there are 1828 unique communities and only 1994 observations.
Intuitively this variable does not add much to the ability to predict violent crime and implementing it would have great downsides such as increased dimensionality which overcomplicates the model for minimum upside.  Therefore, we will drop this variable from consideration.

Furthermore, the documentation for the variable fold reads - "fold: fold number for non-random 10 fold cross validation, potentially useful for debugging, paired tests - not predictive (numeric)".  Clearly this variable should be dropped.

Now states is a bit tricky.  Assume we wanted to apply one-hot encoding.  This would go as follows:

In [110]:
#creating and populating state matrix
unique_states = list(X_clean['state'].apply(lambda x: int(x)).unique())
states_df = np.zeros(shape=(df.shape[0],len(unique_states)))
states_df = pd.DataFrame(states_df, columns=unique_states)

for index, value in enumerate(X_clean['state']):
  states_df.loc[index,value] = 1

assert states_df.sum().sum() == X_clean['state'].shape[0], 'Error in populating matrix'
states_df

,8,53,24,34,42,6,44,21,29,36,...,13,23,38,27,20,56,2,11,32,10
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1989,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1990,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1991,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1992,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [115]:
states_df.sum().sort_values(ascending = False).head(5)

,0
6,278.0
34,211.0
48,156.0
25,121.0
39,109.0


This matrix is not as sparse as if we were to implement one-hot encoding on a variable such as community names.  As we can see the top 5 states are included in over a hundred observations.  However, the issue with adding states_df to the dataset is the high correlation that states has with other variables.  For example, states and percentage of race, or states and income or household size etc are likely to have a high correlation, i.e., the income in a state such as NY is drastically different from other states.

Therefore, it makes sense to drop this variable.

In [132]:
X_final = X_clean.copy()  # Ensure X_final starts as a copy of X_clean

if 'state' in X_final.columns:
    X_final = X_final.drop(columns=['state'])
    print('Successfully dropped - state')

if 'communityname' in X_final.columns:
    X_final = X_final.drop(columns=['communityname'])
    print('Successfully dropped - communityname')

if 'fold' in X_final.columns:
    X_final = X_final.drop(columns=['fold'])
    print('Successfully dropped - fold')

# Ensure exactly 2 columns were dropped
assert X_final.shape[1] == X_clean.shape[1] - 3, 'error'

Successfully dropped - state
Successfully dropped - communityname
Successfully dropped - fold


Now to make sure all data is of the correct type, i.e., the strings containing numbers are floats we run the following code chunk.

In [130]:
for var in X_final.columns:
    if X_clean[var].dtype == 'O':  # Check if the column is an object type (potentially non-numeric)
        try:
            X_clean.loc[:, var] = X_clean[var].astype(float)
        except ValueError:
            print(f"Column {var} contains non-numeric values and cannot be converted.")
    else:
        X_clean.loc[:, var] = X_clean[var].astype(float)

assert X_final.dtypes.unique() == float, 'error in type of data'

In [134]:
X = X_final
X

,population,householdsize,racepctblack,racePctWhite,racePctAsian,racePctHisp,agePct12t21,agePct12t29,agePct16t24,agePct65up,...,PctForeignBorn,PctBornSameState,PctSameHouse85,PctSameCity85,PctSameState85,LandArea,PopDens,PctUsePubTrans,LemasPctOfficDrugUn,OtherPerCap
0,0.19,0.33,0.02,0.90,0.12,0.17,0.34,0.47,0.29,0.32,...,0.12,0.42,0.50,0.51,0.64,0.12,0.26,0.20,0.32,0.36
1,0.00,0.16,0.12,0.74,0.45,0.07,0.26,0.59,0.35,0.27,...,0.21,0.50,0.34,0.60,0.52,0.02,0.12,0.45,0.00,0.22
2,0.00,0.42,0.49,0.56,0.17,0.04,0.39,0.47,0.28,0.32,...,0.14,0.49,0.54,0.67,0.56,0.01,0.21,0.02,0.00,0.28
3,0.04,0.77,1.00,0.08,0.12,0.10,0.51,0.50,0.34,0.21,...,0.19,0.30,0.73,0.64,0.65,0.02,0.39,0.28,0.00,0.36
4,0.01,0.55,0.02,0.95,0.09,0.05,0.38,0.38,0.23,0.36,...,0.11,0.72,0.64,0.61,0.53,0.04,0.09,0.02,0.00,0.51
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1989,0.01,0.40,0.10,0.87,0.12,0.16,0.43,0.51,0.35,0.30,...,0.22,0.28,0.34,0.48,0.39,0.01,0.28,0.05,0.00,0.36
1990,0.05,0.96,0.46,0.28,0.83,0.32,0.69,0.86,0.73,0.14,...,0.53,0.25,0.17,0.10,0.00,0.02,0.37,0.20,0.00,0.23
1991,0.16,0.37,0.25,0.69,0.04,0.25,0.35,0.50,0.31,0.54,...,0.25,0.68,0.61,0.79,0.76,0.08,0.32,0.18,0.91,0.22
1992,0.08,0.51,0.06,0.87,0.22,0.10,0.58,0.74,0.63,0.41,...,0.45,0.64,0.54,0.59,0.52,0.03,0.38,0.33,0.22,0.27


# Step 3: Export data (X,y) to CSV.

In [137]:
data = pd.concat([y,X], axis = 1)

# y == ViolentCrimesPerPop
# X == data - column y

data

,ViolentCrimesPerPop,population,householdsize,racepctblack,racePctWhite,racePctAsian,racePctHisp,agePct12t21,agePct12t29,agePct16t24,...,PctForeignBorn,PctBornSameState,PctSameHouse85,PctSameCity85,PctSameState85,LandArea,PopDens,PctUsePubTrans,LemasPctOfficDrugUn,OtherPerCap
0,0.20,0.19,0.33,0.02,0.90,0.12,0.17,0.34,0.47,0.29,...,0.12,0.42,0.50,0.51,0.64,0.12,0.26,0.20,0.32,0.36
1,0.67,0.00,0.16,0.12,0.74,0.45,0.07,0.26,0.59,0.35,...,0.21,0.50,0.34,0.60,0.52,0.02,0.12,0.45,0.00,0.22
2,0.43,0.00,0.42,0.49,0.56,0.17,0.04,0.39,0.47,0.28,...,0.14,0.49,0.54,0.67,0.56,0.01,0.21,0.02,0.00,0.28
3,0.12,0.04,0.77,1.00,0.08,0.12,0.10,0.51,0.50,0.34,...,0.19,0.30,0.73,0.64,0.65,0.02,0.39,0.28,0.00,0.36
4,0.03,0.01,0.55,0.02,0.95,0.09,0.05,0.38,0.38,0.23,...,0.11,0.72,0.64,0.61,0.53,0.04,0.09,0.02,0.00,0.51
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1989,0.09,0.01,0.40,0.10,0.87,0.12,0.16,0.43,0.51,0.35,...,0.22,0.28,0.34,0.48,0.39,0.01,0.28,0.05,0.00,0.36
1990,0.45,0.05,0.96,0.46,0.28,0.83,0.32,0.69,0.86,0.73,...,0.53,0.25,0.17,0.10,0.00,0.02,0.37,0.20,0.00,0.23
1991,0.23,0.16,0.37,0.25,0.69,0.04,0.25,0.35,0.50,0.31,...,0.25,0.68,0.61,0.79,0.76,0.08,0.32,0.18,0.91,0.22
1992,0.19,0.08,0.51,0.06,0.87,0.22,0.10,0.58,0.74,0.63,...,0.45,0.64,0.54,0.59,0.52,0.03,0.38,0.33,0.22,0.27


In [138]:
data.to_csv('data.csv', index=False)